* Pecu https://github.com/pecu/PyTorch_CSX/blob/master/01_PyTorch_Basic/01_TensorDef.ipynb
* 作者: 莫烦 https://morvanzhou.github.io/tutorials/machine-learning/torch/2-02-variable/

In [2]:
import numpy as np

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Tensor  &  numpy array
* PyTorch tensor 本質上和 numpy array差不多，可以互相轉換。

In [3]:
list1 = [[3,2,1], [4,5,6]]

# list > numpy:
arr1 = np.array(list1)
print(arr1)


# list > pytorch-tensor
tr1 = torch.FloatTensor(list1)
print(tr1)

[[3 2 1]
 [4 5 6]]

 3  2  1
 4  5  6
[torch.FloatTensor of size 2x3]



In [4]:
# numpy => pytorch tensor
tr2 = torch.from_numpy(arr1) # dtype same as numpy
print(tr2)


# pytorch tensor => numpy
arr2 = tr1.numpy()
print(arr2)


 3  2  1
 4  5  6
[torch.LongTensor of size 2x3]

[[ 3.  2.  1.]
 [ 4.  5.  6.]]


# Pytorch Variable
* 在Torch中的 Variable 就是一個存放可變化數值的位置，裡面值可不停的變化，如果用一個變量進行計算，那結果也是一個同類型的變量。
* 變量計算時，會默默搭建計算圖將所有的計算步驟連接起來，最後進行誤差反向傳遞的時候，一次性將所有變量裡面的微分值都計算出來。

In [5]:
# 先產生數值
tensor = torch.FloatTensor([[1,2],[3,4]])


# 把值放到變數籃子裡 ( requires_grad表示是否參與誤差反向傳播 & 要不要計算梯度 )
variable = Variable(tensor, requires_grad=True)
variable

Variable containing:
 1  2
 3  4
[torch.FloatTensor of size 2x2]

In [6]:
# 變數做計算
v_out = torch.mean(variable * variable)   
print(v_out)

Variable containing:
 7.5000
[torch.FloatTensor of size 1]



In [7]:
v_out.backward() 
print(variable.grad)    # 初始 Variable 的梯度
# v_out = (1/4) * sum(variable * variable)  =>  v_out 對 variable 微分 ＝ 1/2 (variable) #次方下乘,次方減 1

Variable containing:
 0.5000  1.0000
 1.5000  2.0000
[torch.FloatTensor of size 2x2]



In [8]:
# 變數很多運算做不了，可以用 variable.data 取出數值（變成 Tensor）
print(variable.data)  


 1  2
 3  4
[torch.FloatTensor of size 2x2]



# Review: Embedding
* 將一組數字轉成一組向量，且數字與向量嚴格一對一對應。

In [9]:
# 有一組字串
'I love machine learning !'

# 字典化(我們只能處理數字)
word_to_idx = {'I': 0, 'love': 1, 'machine':2, 'learning':3, '!':4}  

# 張量
word_tensor = torch.LongTensor([word_to_idx["I"]])

# 裝進 Variable
word_var = Variable(word_tensor)

# 把這個 Variable 變成一串數字 
embeds = nn.Embedding(1, 10)
word_embed = embeds(word_var)

print(word_embed)

Variable containing:
 0.6614  0.2669  0.0617  0.6213 -0.4519 -0.1661 -1.5228  0.3817 -1.0276 -0.5631
[torch.FloatTensor of size 1x10]



In [16]:
# 一次將一句話向量化(裝在變數裡)
word_tensor = torch.LongTensor([ word_to_idx["I"],
                                 word_to_idx["love"],
                                 word_to_idx["machine"],
                                 word_to_idx["learning"],
                                
                                 word_to_idx["learning"] # 重複
                               ])

# 裝進 Variable
word_var = Variable(word_tensor)

# 把這個 Variable 變成一串數字 
embeds = nn.Embedding(5, 10)
word_embed = embeds(word_var)

print(word_embed)

Variable containing:
 0.3711 -0.1741  1.1385 -1.4325  0.2623  1.2968  1.0816  0.4075 -0.9404 -1.3203
 1.6002  1.3146 -0.6118 -0.9419 -0.1675 -1.6990 -2.0724  1.5600 -0.5075 -1.6533
-0.0907 -1.0677 -0.4728 -0.0388 -0.0063 -0.1100  0.1423  0.2453 -0.6245 -0.7920
 1.2385  0.8845 -0.4387 -0.9479 -1.0857 -0.0113  0.5639 -0.5461 -0.5868 -0.1270
 1.2385  0.8845 -0.4387 -0.9479 -1.0857 -0.0113  0.5639 -0.5461 -0.5868 -0.1270
[torch.FloatTensor of size 5x10]



In [23]:
# 常犯錯誤：

# 把一個 index 改成 6 則超出原本 nn.Embedding(5, 10) [0,1,2,3,4] 的範圍
word_to_ix = {'I': 0, 'love': 1, 'machine':2, 'learning':3, '!':4}  

# 一次將一句話向量化(裝在變數裡)
word_tensor = torch.LongTensor([ word_to_ix["I"],
                                 word_to_ix["love"],
                                 word_to_ix["machine"],
                                 word_to_ix["learning"],
                                 word_to_ix["!"]
                               ])



word_var = Variable(word_tensor)
# 把這個 Variable 變成一串數字 
embeds = nn.Embedding(5, 10)
word_embed = embeds(word_var)

print(word_embed)

Variable containing:
 0.7491 -1.5348  0.3031  0.4657 -1.7558 -0.2372  0.8212 -0.9065  0.3359  1.4863
 0.1520 -1.3443  0.6541  2.2225  0.2947 -0.8987  0.6914 -0.3367 -0.9926  0.4029
 0.4275 -1.1540 -0.1806  1.4955  0.7525 -0.3977 -1.2727 -0.1921  0.6353 -1.3944
-0.9095  0.1111 -0.0352 -0.8154  0.2774  0.6695  0.9860  0.1008  0.7559 -0.8964
 0.7890  0.6291 -0.5083 -0.3679 -0.9695 -0.3280 -0.6444 -0.1242  0.1136  1.0938
[torch.FloatTensor of size 5x10]



## 順便名詞解釋：
##### 詞向量
詞向量(word vector）是近年在自然語言處理中被廣泛使用的一種技術。一言以蔽之，我們想使用一個向量來表示每一個詞，如此一來，就能把一段由許多詞組成的文句，轉換成一個個詞向量來表示，並把這樣數值化的資料，送到模型裡做後續的應用。

###### one-hot representation 直接表示法：
最直觀的詞表示方法, 把每個詞表示為一個很長的向量，維度是詞表大小，其中大多數元素為0，只有一個維度的值為1代表了當前的詞。<br>
本方式無法表格每個詞的遠近關係。<br>
ex: 上面的 'I' =    [1, 0, 0, 0, 0]
ex: 上面的 'love' = [0, 1, 0, 0, 0]





###### word embedding 結合神經網絡訓練：
透詞直接向量化後去配合模型訓練後，讓模型自己找出最好的向量表示方式。<br>
可以解決one-hot 維度過大的缺點，將原來稀疏的巨大維度壓縮嵌入到一個更小維度的空間。<br>




###### word2vec 語言模型：
Word2Vec是從大量文本語料中學習語義知識的一種模型，目的想衣文本學到更好的向量化表示。<br>
Word2Vec = Continuous Bag Of Words（CBOW）+ k-skip-N-gram <br>
* k-skip-N-gram: 輸入目標詞詞向量，預測出可能的前後詞<br> 
* CBOW: 輸入前後詞(contex word)詞向量，要預測出可能的目標詞(target word)



# Pytorch 兩個函數說明(Attention 會用到)

In [30]:
# Applies a linear transformation to the incoming data: y = Ax + b
m = nn.Linear(5, 10, bias=True)


input = Variable(torch.FloatTensor([1,2,3,4,5]))
output = m(input)
print(output)

Variable containing:
 3.3969
-3.5469
 0.3958
-0.0856
 0.3659
-1.8985
 0.3397
 0.1151
 1.2896
-4.5139
[torch.FloatTensor of size 10]



In [13]:
# torch.bmm(很多矩陣相乘)
batch1 = torch.randn(10, 3, 4) # 10 個 3X4
batch2 = torch.randn(10, 4, 5) # 10 個 4X5


res = torch.bmm(batch1, batch2)
res.size()# 10 個 3X5

torch.Size([10, 3, 5])